to get this notebook working: 

```
poetry add ipykernel hvplot shapely searvey geoviews utide
```

In [ ]:
import searvey
import shapely
import utide
import pandas as pd
import hvplot.pandas

In [ ]:
# small function to detide signal (using Utide: https://github.com/wesleybowman/UTide)
def surge(ts: pd.Series, lat: float, resample: int = None): 
    ts0 = ts.copy()
    OPTS = {
        "constit": "auto", 
        "method": "ols", 
        "order_constit": "frequency",
        "Rayleigh_min": 0.97,
        "lat": lat,
        "verbose": True
    }
    if resample is not None:
        ts = ts.resample(f"{resample}min").mean()
        ts = ts.shift(freq=f"{resample / 2}min")
        coef = utide.solve(ts.index, ts, **OPTS)
        tidal = utide.reconstruct(ts0.index, coef, verbose = OPTS["verbose"])
        return pd.Series(data=ts0.values - tidal.h, index = ts0.index)

In [ ]:
ioc_df = searvey.get_ioc_stations()
africa = shapely.box(-26, -35, 63, 38)
ioc_africa = ioc_df[ioc_df.geometry.within(africa)]
ioc_africa.hvplot(tiles=True, hover_cols = ['ioc_code'])


example for `zanz`: 

In [ ]:
station = "zanz"
sensor = "prs"

In [ ]:
raw = searvey.fetch_ioc_station(
    station, 
    pd.Timestamp.now()-pd.Timedelta(days = 365), 
    pd.Timestamp.now()
)
raw.describe()

In [ ]:
_lat = ioc_df[ioc_df.ioc_code == station].lat.values[0]

In [ ]:
detided = surge(raw[sensor], lat = _lat, resample=2)

visualise the raw signal

In [ ]:
raw[sensor].sample(frac=1/5).dropna().hvplot().opts(width=1300, height = 900)

visualise the detided signal (without tidal components)

In [ ]:
detided.sample(frac=1/5).hvplot(ylim = (-0.5, 0.5)).opts(width=1300, height = 900)